# Detecção de Infarto do miocárdio usando sinais ECG
O presente documento é a representação da rede neural do artigo: "Application of deep convolutional neural network for automated detection of myocardial infarction using ECG signals".
<br>
Retirado em: <a href= "https://www.sciencedirect.com/science/article/pii/S0020025517308009"> Application of deep convolutional neural network for automated detection of myocardial infarction using ECG signals </a>
## Artigo
O artigo utiliza redes neurais convolucionais para detectar automaticamente infarto do coração. Dois modelos diferentes foram utilizados, um com ruído e outro sem. Aqui reproduziremos a arquitetura com os ruídos.
## Database
Os dados foram retirados de: <a href= "https://www.physionet.org/content/ptbdb/1.0.0/"> PTB Diagnostic ECG Database </a>. Apesar de ter 9 classes, somente 2 foram utilizadas, as de pessoas saudáveis (52) e de infarto do miocárdio(148). Outro fator importante é que são utilizados 12 leads para classificação, entretanto somento o lead II é usado. Por último, a frequência de obtenção de sinais é de 2KHz.

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch

## Carregar os dados

In [9]:
# Carregar a tabela csv
df = pd.read_csv("batimentos.csv", header= None)
df.head


<bound method NDFrame.head of             0         1         2         3         4         5         6    \
0      436.0000  426.0000  422.0000    0.4200  416.0000  418.0000    0.4175   
1      444.0000  458.0000  455.0000    0.4475    0.4485  443.0000  439.0000   
2      436.0000  441.0000  436.0000  432.0000  432.0000    0.4235    0.4200   
3      407.0000  404.0000    0.4135    0.4205    0.4185  413.0000  414.0000   
4        0.4095    0.3965  404.0000    0.4295  437.0000    0.4375  435.0000   
...         ...       ...       ...       ...       ...       ...       ...   
18179    0.1255    0.1165    0.1095  105.0000    0.1015    0.0995    0.1035   
18180   43.0000   61.0000    0.0475   47.0000   85.0000    0.0800    0.0735   
18181    0.1025    0.1155  103.0000    0.0915  106.0000    0.1105    0.0775   
18182    0.1135   71.0000    0.0555    0.0635    0.0715    0.0875   77.0000   
18183    0.1105    0.1015    0.0995   89.0000   77.0000    0.0785    0.0855   

            7        

In [27]:
labels = ["Positive", "Negative"]
negative = 0
positive = 0

#for x in (df.iloc[:,[651]]):
#    print(x)
    #if x == 0:
    #    negative =+ 1
    #else:
    #    positive =+ 1

print(type(df.iloc[:, [651]]))
#print(negative)
#print(positive)
#fig, ax = plt.subplots()
#ax.bar()

<class 'pandas.core.frame.DataFrame'>


In [28]:
type(df)

pandas.core.frame.DataFrame

In [6]:
total_size = df.shape[0]

train_size = int(0.63 * total_size)
validation_size = int(0.27 * total_size)
test_size = total_size - (validation_size + train_size)

print(f"Train size: {train_size}, Validation size: {validation_size},\
 Test size: {test_size}")


Train size: 11455, Validation size: 4909, Test size: 1820


In [ ]:
# Dividir o data set em treino teste e validation
from torch.utils.data import DataLoader
# Dividir o dataset em treino teste e validation 


## Arquitetura

In [ ]:
model = 

## Treino

## Teste